# Project: Data Modelling with Apache Cassandra


We start the project with overall import statements for the needed Python libraries, followed by code implementation for the specific tasks.

#### Import Python packages 

In [ ]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

from cassandra.cluster import Cluster

# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Creating list of filepaths to process original event csv data files

In [ ]:
# checking your current working directory
print(os.getcwd())

filepath = os.getcwd() + '/event_data'
for root, dirs, files in os.walk(filepath):  
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [ ]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print("\033[1;34;43mInformation about Event Data File:\033[0;0m")
print(f'\033[1;30mTotal number of rows:\033[0;0m\n{len(full_data_rows_list)}')
# uncomment the code below if you would like to check to see what the list of event data rows will look like
print(f'\n\033[1;30mFirst two event data rows:\033[0;0m\n- {full_data_rows_list[0]}\n- {full_data_rows_list[1]}')

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [ ]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [ ]:
# This should make a connection to a Cassandra instance of your local machine:
# (127.0.0.1)
# Then, to establish connection and begin executing queries, we need a session

try: 
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [ ]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [ ]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data
The three queries we have to create tables for are:

1. **Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4**


2. **Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182**
    

3. **Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'**


### Query 1
Give me 
- the artist,
- song title and 
- song's length in the music app history 

and regarding validation, that was heard during
- sessionId = 338 and 
- itemInSession = 4

In [1]:
# Query creation,  
# note: column names are given with .csv file, see image example above

query = "CREATE TABLE IF NOT EXISTS song_library "
query = query + "(sessionId int, \
                  itemInSession int, \
                  artist text, \
                  song text, \
                  length float, \
                  PRIMARY KEY (sessionId, itemInSession))"
 
try:
    session.execute(query)
except Exception as e:
    print(e) 

In [ ]:
# Insert the data into our song_library
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Assign the INSERT statements into the `query` variable      
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        query = "INSERT INTO song_library (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[10], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [ ]:
# Transformation validation
val_query = "SELECT artist, song, length FROM song_library WHERE sessionId=338 AND itemInSession=4"

try:
    rows = session.execute(val_query)
except Exception as e:
    print(e)
    
for row in rows:
    print(f'Artist: {row.artist}, Song title: "{row.song}", Song length: {round(row.length, 4)})

### Query 2
Give me only the following: 
- name of artist
- song (sorted by itemInSession; must be part of the primary key)
- user (first and last name)

and regarding validation: 
- for userid = 10 and
- sessionid = 182

In [ ]:
# Query creation
query = "CREATE TABLE IF NOT EXISTS user_library "
query = query + "(userId int, \
                  sessionId int, \
                  itemInSession int, \
                  artist text, \
                  song text, \
                  firstName text, \
                  lastName text, \
                  PRIMARY KEY (userId, sessionId, itemInSession))"
   
try:
    session.execute(query)
except Exception as e:
    print(e)      

In [ ]:
# Insert data into user_library table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO user_library (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [ ]:
# Transformation validation
val_query = "SELECT artist, song, firstName, lastName FROM user_library WHERE userId=10 AND sessionId=182"
try:
    rows = session.execute(val_query)
except Exception as e:
    print(e)
for row in rows:
    # print(row)
    print(f'Artist: {row.artist}, Song title: "{row.song}", User first and last name: {row.firstname} {row.lastname}')

### Query 3
Give me every user name (first and last) in my music app history who listened to the song <i>'All Hands Against His Own'</i>.

In [ ]:
# Query creation
query = "CREATE TABLE IF NOT EXISTS song_users_library "
query = query + "(song text, \
                  userId int, \
                  firstName text, \
                  lastName text, \
                  PRIMARY KEY (song, userId))"
 
try:
    session.execute(query)
except Exception as e:
    print(e)           

In [ ]:
# Insert data into song_users_library table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO song_users_library (song, userId, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [ ]:
# Transformation validation
val_query = "SELECT * FROM song_users_library WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(val_query)
except Exception as e:
    print(e)
for row in rows:
    # print (row)
    print(f'Song title: "{row.song}", User id, first and last name: {row.userid}, {row.firstname} {row.lastname}')

### Drop the tables before closing out the sessions

In [4]:
# Final table clean up
query = "DROP TABLE song_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE user_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE song_users_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection

In [ ]:
session.shutdown()
cluster.shutdown()